In [ ]:
import os
import pandas as pd
import mysql.connector
from  mysql.connector import errorcode
import ast

In [ ]:
def limpieza_datos_detalles_actores():
    """
    Esta función toma una archivo csv, lo pasa a lista de tuplas y comprueba que no haya elementos nan, null o duplicados.
    
    Args:
    - esta función no recibe ningún parametro

    Returns:
    - esta funcion devuelve una lista de tuplas con los datos de los actores listos para insertar en la bbdd
    """

    # Lista a usar durante la limpieza
    lista_tuplas_detalles = []
    lista_final_detalles = []
    lista_actores = []
    
    for i in range(1990,2024):
        for j in range(1,3):
            try:
                df_detalles_actores= pd.read_csv(f"datos/datos_actores/lista_actores_{i}_1000_{j}000.csv")
            except:
                print(f"lista_actores_{i}_1000_{j}000.csv - no existe")
                break

    #Se convierten las listas en formato string provenientes del csv en una string (eliminando la lista)
    for fila in df_detalles_actores.values:
    
        try:
            conocido = ast.literal_eval(fila[2])
        
        except:
            conocido= "No data"
        try:    
            que_hace = ast.literal_eval(fila[3])
            que_hace=", ".join(que_hace)
        except:
            que_hace = "No data"
        lista_tuplas_detalles.append((fila[0], fila[1], conocido[0], que_hace, fila[4]))

    lista_tuplas_detalles = set(lista_tuplas_detalles)
    
    # Se crea una lista de todos los actores en la lista de tuplas para extraer los actores que estan repetidos como nombre de interprete, pero que por algún error durante la extracción, el resto de la tupla no es igual (en next steps se deberan de volver a buscar)
    for tupla in lista_tuplas_detalles:
        lista_actores.append(tupla[0])

    duplicados_actores = [actor for actor in lista_actores if lista_actores.count(actor)>1]

    for tupla in lista_tuplas_detalles:
        if tupla[0] not in set(duplicados_actores): 
            if not pd.isnull(tupla[0]):
                for elemento in tupla:
                    if elemento != elemento or pd.isnull(elemento):
                        elemento = "No data"
                        print(f"elemento: {elemento}")
                lista_final_detalles.append(tupla)   

    return lista_final_detalles

In [ ]:
def insertar_detalles_actores(lista):
    """
    Esta función toma una lista de tuplas y la inserta en una tabla de la bbdd
    
    Args:
    - lista: listade tuplas donde cada tupla incluye los datos de un actor: nombre actor, año nacimiento, conocido por, que hace, premios oscar.

    Returns:
    - esta funcion devuelve el número de datos insertados

    """

    contrasena = os.environ.get('contraseña')

    # Conexion con la base de datos e inserción de los datos finales de los detalles de los interpretes
    cnx = mysql.connector.connect(user="root",
                                password=contrasena,
                                host="localhost", database = "bho_peliculas")

    mycursor = cnx.cursor()

    sql_detalles = "INSERT INTO datos_interpretes (nombre_interprete, año_nacimiento, se_conoce_por, que_hace, premios) VALUES (%s, %s, %s, %s, %s)"

    try:
        mycursor.executemany(sql_detalles, lista[:-1])
        cnx.commit()
        print(mycursor.rowcount, "registros insertados")
            
    except mysql.connector.Error as err:
        print(err)
        print("Error Code:", err.errno)
        print("SQLSTATE", err.sqlstate)
        print("Message", err.msg)
        print("peliculas - error inserción")

    cnx.close()    

    return mycursor.rowcount

In [ ]:
#llamar a las funciones para limpiar los datos e insertarlos en la bbdd
insertar_detalles_actores(limpieza_datos_detalles_actores())     